In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape

## load in damage data 

In [2]:
# load in damage files
path_ian_70_buildings = 'data/Hurricane_Ian_scenarios/Hurricane_Ian_Shifted_70/impacts/Impacts_building_footprints_Hurricane_Ian_Shifted_70.gpkg'
all_ian_70_buildings = gpd.read_file(path_ian_70_buildings)

In [3]:
#load in damage files
path_ian_buildings = 'data/Hurricane_Ian_scenarios/Hurricane_Ian/impacts/Impacts_building_footprints_Hurricane_Ian.gpkg'
all_ian_buildings = gpd.read_file(path_ian_buildings)

In [4]:
# select only the residential buildings
ian_70_buildings = all_ian_70_buildings.loc[all_ian_70_buildings['Primary Object Type'] == 'RES']
all_ian_buildings = all_ian_buildings.loc[all_ian_buildings['Primary Object Type'] == 'RES']

## load in EAD data 

In [5]:
impacts_detailed_path_geo = "data/current_risk_no_measures/Impacts/Impacts_building_footprints_current_risk_no_measures.gpkg"
impacts = gpd.read_file(impacts_detailed_path_geo)

In [6]:
# select only the residential buildings
impacts_res = impacts.loc[impacts['Primary Object Type'] == 'RES']

Only run following cells to retrieve shapefile for charleston

In [7]:
# # Access shapefile of South Carolina census blockgroup
# SC_bg = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2021/BG/tl_2021_45_bg.zip")

# # Reproject shapefile to UTM Zone 17N
# # https://spatialreference.org/ref/epsg/wgs-84-utm-zone-17n/
# SC_bg = SC_bg.to_crs(epsg = 4326)#32617)

In [8]:
# # Drop multiple columns
# columns_to_drop = ['AWATER','MTFCC','ALAND','NAMELSAD','INTPTLAT','INTPTLON']#'STATEFP','COUNTYFP', 'TRACTCE','FUNCSTAT'
# clean_SC_bg = SC_bg.drop(columns=columns_to_drop)#,'NAMELSAD'

In [9]:
# clean_SC_bg.info()

In [10]:
# shape_charleston = clean_SC_bg[clean_SC_bg['COUNTYFP'] == '019']
# shape_charleston['GEOID'] = shape_charleston['GEOID'].astype(int)
# shape_charleston.info()

In [11]:
# shape_charleston.to_file('data/shape_charleston.gpkg')

In [12]:
shape_charleston = gpd.read_file('data/shape_charleston.gpkg')

In [13]:
# census_bg_path = 'data/census_bg_deltares.gpkg'
# census_bg = gpd.read_file(census_bg_path)

In [14]:
# gdf1 = census_bg
# gdf2 = shape_charleston

In [15]:
# if gdf1['geometry'].equals(gdf2['geometry']):
#     print("The shapefiles have the same geometry.")
# else:
#     print("The shapefiles have different geometry.")

In [16]:
# add geometry to floodadapt data by joining the files
join_shape_buildings_ian_70 = ian_70_buildings.sjoin(shape_charleston, how='inner')
join_impacts = impacts_res.sjoin(shape_charleston, how='inner')

In [17]:
max_damage = {'Max Potential Damage: Structure': 'sum'}
damage_str = {'Damage: Structure':'sum'}
EAD = {'Risk (EAD)': 'sum'}

In [18]:
df_max_damage = join_shape_buildings_ian_70.groupby(join_shape_buildings_ian_70['GEOID']).aggregate(max_damage)
df_damage_str = join_shape_buildings_ian_70.groupby(join_shape_buildings_ian_70['GEOID']).aggregate(damage_str)
df_EAD = join_impacts.groupby(join_impacts['GEOID']).aggregate(EAD)

In [19]:
data_frames = [df_max_damage, df_damage_str,df_EAD]
df_damage = pd.concat(data_frames, axis=1, ignore_index=False, join='inner')

In [20]:
# Perform the join with shapefiles
new = shape_charleston.merge(df_damage, on = "GEOID")

In [21]:
#further clean the dataset
columns_to_drop = ['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','FUNCSTAT']
df_damage_clean = new.drop(columns=columns_to_drop)
df_damage_clean.head(2)

,GEOID,geometry,Max Potential Damage: Structure,Damage: Structure,Risk (EAD)
0,450190046193,"POLYGON ((-79.86198 32.79452, -79.86196 32.794...",1.125926e+08,0.0,22586.35
1,450190031171,"POLYGON ((-80.06813 32.96169, -80.06798 32.961...",7.671196e+07,0.0,10281.18


In [22]:
df_damage_clean.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   GEOID                            252 non-null    int64   
 1   geometry                         252 non-null    geometry
 2   Max Potential Damage: Structure  252 non-null    float64 
 3   Damage: Structure                252 non-null    float64 
 4   Risk (EAD)                       252 non-null    float64 
dtypes: float64(3), geometry(1), int64(1)
memory usage: 11.8 KB


In [23]:
################
# df_damage_clean.to_csv('data/damage_shape_OG.csv', index=False)
# df_damage_clean.to_csv('data/damage_shape_25.csv', index=False)
df_damage_clean.to_csv('data/damage_shape.csv', index=False)